In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


In [2]:
Sample_sub = pd.read_csv('/kaggle/input/playground-series-s5e7/sample_submission.csv')
train_set = pd.read_csv('/kaggle/input/playground-series-s5e7/train.csv')
test_set = pd.read_csv('/kaggle/input/playground-series-s5e7/test.csv')

In [3]:
print(Sample_sub.head())
# print(train_set.head())
print(test_set.head())

      id Personality
0  18524   Extrovert
1  18525   Extrovert
2  18526   Extrovert
3  18527   Extrovert
4  18528   Extrovert
      id  Time_spent_Alone Stage_fear  Social_event_attendance  Going_outside  \
0  18524               3.0         No                      7.0            4.0   
1  18525               NaN        Yes                      0.0            0.0   
2  18526               3.0         No                      5.0            6.0   
3  18527               3.0         No                      4.0            4.0   
4  18528               9.0        Yes                      1.0            2.0   

  Drained_after_socializing  Friends_circle_size  Post_frequency  
0                        No                  6.0             NaN  
1                       Yes                  5.0             1.0  
2                        No                 15.0             9.0  
3                        No                  5.0             6.0  
4                       Yes                  1.0    

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [4]:
train_set.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


In [5]:
#clearning the train_Set
data_clean = train_set.dropna()

In [6]:
# One-hot encode all categorical features (including numerically-stored ones)
categorical_columns = ['Stage_fear', 'Drained_after_socializing', 'Social_event_attendance', 'Friends_circle_size']
data_encoded = pd.get_dummies(data_clean, columns=categorical_columns, drop_first=True)

# Encode the target
label_encoder = LabelEncoder()
data_encoded['Personality'] = label_encoder.fit_transform(data_encoded['Personality'])

# Features and target
X = data_encoded.drop('Personality', axis=1)
y = data_encoded['Personality']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Evaluation
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

              precision    recall  f1-score   support

   Extrovert       0.97      0.98      0.97      1700
   Introvert       0.88      0.86      0.87       338

    accuracy                           0.96      2038
   macro avg       0.93      0.92      0.92      2038
weighted avg       0.96      0.96      0.96      2038



In [7]:
#Prepare test data (copy and encode as before)
test_data_copy = test_set.copy()
test_encoded = pd.get_dummies(test_data_copy, columns=categorical_columns, drop_first=True)
#Align columns with training data
for col in X.columns:
    if col not in test_encoded.columns:
        test_encoded[col] = 0
test_encoded = test_encoded[X.columns]

#Impute missing values (replace NaNs with mean)
imputer = SimpleImputer(strategy='mean')
test_encoded_imputed = imputer.fit_transform(test_encoded)

#Predict
y_test_pred = model.predict(test_encoded_imputed)

#Inverse label encoding
y_test_labels = label_encoder.inverse_transform(y_test_pred)

#Prepare submission
submission = pd.DataFrame({
    'id': test_set['id'],
    'Personality': y_test_labels
})

submission.to_csv('submission.csv', index=False)
print(submission.head())

      id Personality
0  18524   Extrovert
1  18525   Introvert
2  18526   Extrovert
3  18527   Extrovert
4  18528   Introvert


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
